# Pi calculation






В ходе выполнения лабораторной работы был реализован алгоритм рассчета числа π методом Монте-Карло с применением CUDA.

**Входные данные:** количество точек N.

**Выходные данные:** 

*   время выполнения программы на GPU и CPU;
*   число π, рассчитанное на GPU и CPU.

In [1]:
import numpy as np
from numba import cuda
from time import time
import random
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32

In [2]:
N = 1000000

### Реализация на GPU

In [3]:
@cuda.jit
def calc_pi_numba(rng_states, N, res):
    idx = cuda.grid(1)
    M = 0
    for i in range(N):
        # генерация случайных чисел с плавающей точкой
        x = xoroshiro128p_uniform_float32(rng_states, idx)
        y = xoroshiro128p_uniform_float32(rng_states, idx)
        if x ** 2 + y ** 2 <= 1.0:
            M += 1
    res[idx] = 4 * M / N

In [10]:
threads_per_block = 32
blocks = 32

rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=1)
out = np.zeros(threads_per_block * blocks, dtype=np.float32)

out_gpu = cuda.to_device(out)

start = time()
calc_pi_numba[blocks, threads_per_block](rng_states, N, out_gpu)
time_gpu = time() - start

pi_gpu = out_gpu.copy_to_host().mean()

### Реализация на CPU

In [5]:
def calc_pi(N):
    M = 0
    for i in range(N):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        if x ** 2 + y ** 2 < 1.0:
            M += 1
    return 4 * M / N

In [11]:
start = time()
pi_cpu = calc_pi(N)
time_cpu = time() - start

### Сравнение работы алгоритмов на GPU и CPU

In [12]:
print('-------------GPU-------------')
print('pi:', out_gpu.copy_to_host().mean(), 'time:', time_gpu)
print('')
print('-------------CPU-------------')
print('pi:', pi_cpu, 'time:', time_cpu)
print('')
print('Сравнение с числом pi:')
print('GPU:', abs(np.pi -  pi_gpu) )
print('CPU:', abs(np.pi -  pi_cpu) )

-------------GPU-------------
pi: 3.1415448 time: 0.0009541511535644531

-------------CPU-------------
pi: 3.14164 time: 0.9388682842254639

Сравнение с числом pi:
GPU: 4.783471161928787e-05
CPU: 4.734641020709418e-05


Таким образом, вычисление числа pi на GPU происходит значительно быстрее, чем на CPU. 